## Structure of a Waiting Line System 

*Information from:
**Quantitative Methods for Business**,
Twelfth Edition
David R. Anderson, Dennis J. Sweeney,
Thomas A. Williams, Jeffrey D. Camm,
James J. Cochran, Michael J. Fry,
Jeffrey W. Ohlmann*


To illustrate the basic features of a waiting line model, we consider the waiting line at the Burger Dome fast-food restaurant. Burger Dome sells hamburgers, cheeseburgers, french fries, soft drinks, and milk shakes, as well as a limited number of specialty items and dessert selections. Although Burger Dome would like to serve each customer immediately, at times more customers arrive than can be handled by the Burger Dome food service staff. Thus, customers wait in line to place and receive their orders.

Burger Dome is concerned that the methods currently used to serve customers are resulting in excessive waiting times and a possible loss of sales. Management wants to conduct a waiting line study to help determine the best approach to reduce waiting times and improve service.

### Single-Server Waiting Line 

In the current Burger Dome operation, an employee takes a customer’s order, determines the total cost of the order, receives payment from the customer, and then fills the order. Once the first customer’s order is filled, the employee takes the order of the next customer waiting for service.

This operation is an example of a single-server waiting line. Each customer entering the Burger Dome restaurant is served by a single order-filling station that handles order placement, bill payment, and food delivery. When more customers arrive than can be served immediately, they form a waiting line and wait for the order-filling station to become
available. 


![im](https://raw.githubusercontent.com/Izainea/visualizacion/master/img/Captura%20de%20pantalla%20de%202020-11-12%2010-29-49.png)

## Distribution of Arrivals
Defining the arrival process for a waiting line involves determining the probability distribution for the number of arrivals in a given period of time. For many waiting line situations,the arrivals occur randomly and independently of other arrivals, and we cannot predict when an arrival will occur. In such cases, analysts have found that the Poisson probability distribution provides a good description of the arrival pattern.

### Exercise
Determine the parameters of the Poisson Distribution for the following data on the entry of Customers in Burger Dome.

# Packages installation

In [ ]:
!pip install pymc3==3.8

     |████████████████████████████████| 911kB 10.3MB/s 
     |████████████████████████████████| 1.5MB 41.8MB/s 
     |████████████████████████████████| 4.3MB 45.8MB/s 
     |████████████████████████████████| 727kB 32.6MB/s 
     |████████████████████████████████| 296kB 48.4MB/s 
  Found existing installation: xarray 0.15.1
    Uninstalling xarray-0.15.1:
      Successfully uninstalled xarray-0.15.1
  Found existing installation: pymc3 3.7
    Uninstalling pymc3-3.7:
      Successfully uninstalled pymc3-3.7


# Data Exploration

In the data base we can find the arrival register in one day in the Burger Dome. We assume that the meditions are mainly correct, then the times that are not in the data base will are take as moments without arrivals or moments with zero arrivals. Thus, we can take an indicator that counts the number of arrivals by an given intervals of time.

In [ ]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd

# scipy for distributions and algorithms
import scipy
from scipy import optimize
import scipy.stats as ss

# pymc3 for model implementation
import pymc3 as pm
import theano.tensor as tt

# plotting
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
from IPython.core.pylabtools import figsize

matplotlib.rcParams['figure.figsize'] = (10, 3)
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['ytick.major.size'] = 20
plt.style.use("seaborn-deep")

# data reading
Data = pd.read_csv('https://raw.githubusercontent.com/Izainea/visualizacion/master/Clients.csv',index_col='Unnamed: 0')

In [ ]:
# timeline construction
Data1=np.array(Data)
time = []
for i in range(7,20):     # takes the range of time from 7:00 am to 20:00 pm
  for j in range(60):     # Hours of 60 minutes
    for k in range(60):   # Minutes of 60 seconds
      if i<10:
        if j<10:
          if k<10:
            time.append("0"+str(i)+":0"+str(j)+":0"+str(k))
          else:
            time.append("0"+str(i)+":0"+str(j)+":"+str(k))
        else:
          if k<10:
            time.append("0"+str(i)+":"+str(j)+":0"+str(k))
          else:
            time.append("0"+str(i)+":"+str(j)+":"+str(k))
      else:
        if j<10:
          if k<10:
            time.append(str(i)+":0"+str(j)+":0"+str(k))
          else:
            time.append(str(i)+":0"+str(j)+":"+str(k))
        else:
          if k<10:
            time.append(str(i)+":"+str(j)+":0"+str(k))
          else:
            time.append(str(i)+":"+str(j)+":"+str(k))

# The indicator of arrivals
indicador = []
for i in time:
  if i in Data1:
    indicador.append(1)   # As the all arrivals are in different times, just take 1 arrival in the true case
  else:
    indicador.append(0)

Data = pd.DataFrame({"Tiempo":time,"Indicador":indicador})
Data[370:375]

![texto del enlace](https://photos.google.com/album/AF1QipMrxzBSFWr4qZGlHDVDdXIJRZaFSQNC-o5E00sw/photo/AF1QipPgVNWctXD4YTwY-CjXo7RLR6NsmrVVyCW2q5Vc)

In [ ]:
# The time intervals construction
n = 1   # length of intervals in minutes
m = int(12*60/n)
k = 0
lim = [k]
for i in range(m):
  k += n
  if k >= 60:
    break
  lim.append(k)

# Writing of extremals of the intervals
time1 = []
for i in range(7,20):
  for j in lim:
    if i<10:
      if j<10:
        time1.append("0"+str(i)+":0"+str(j)+":00")
      else:
        time1.append("0"+str(i)+":"+str(j)+":00")
    else:
      if j<10:
        time1.append(str(i)+":0"+str(j)+":00")
      else:
        time1.append(str(i)+":"+str(j)+":00")

## Arrivals Data

Using a histogram, we can describe correctly the data of arrivals in the choose time intervals.

In [ ]:
# The lists of arrivals by time interval
time_boxes = []   # times
indi_boxes = []   # arrivals

for i in range(len(time1)-1):
  A = []   # time interval 
  B = []
  for j in range(len(time)):
    if time1[i] < time[j] < time1[i+1]:   # if the medition are in the interval return True
      A.append(time[j])
      B.append(indicador[j])
  time_boxes.append(A)
  indi_boxes.append(B)

# Number of arrivals by time interval
counts = []
for i in indi_boxes:
  counts.append(sum(i))

print("The number of intervals are %d" %len(counts))

The number of intervals are 779

In [ ]:
# number of clients for probability prediction
arrivals_est = range(7)

# Data for bar plots
count_norm = []
for i in range(len(arrivals_est)):
  k = counts.count(i)
  count_norm.append(k/len(counts))

figsize(16,6)

# Histogram of arrivals in the Burger Dome
plt.title("Bar plot for buyer income")
plt.xlabel("Number of buyers in %d minute intervals" %n)
plt.ylabel("Frequency")
#plt.hist(counts, density=True,bins=5)
plt.bar(range(7),count_norm,label="obs")
plt.ylim(top=1)
plt.show()

![texto del enlace](https://docs.google.com/drawings/d/e/2PACX-1vT5lY50lYyqErYGvbiThZT9UHlTSsVEq3A1uNwDwixMOeMnWIx1gnFDerqxofws7nrLJYfC3n6finVw/pub?w=900&h=377)

## Poisson Distribution

It seems that we can model the number of arrivals with a Poisson distribution, let's remember that the Poisson distribution is a discrete distribution with parameter $\lambda$, where $\lambda$ represents the mean number of ocurrences of the event in a given time interval (note that $\lambda\in(0,\infty)$). The density function of this distribution is given by

$$P_\lambda(x)=\left\{\begin{array}{cl}
\frac{e^{-\lambda}\lambda^x}{x!} & \text{if }x=0,1,2,\cdots \\
0 & \text{in other case}
\end{array}\right.$$

In [ ]:
x = range(0, 25)   # example domain of P
lam = (0.5,4,8,15)   # example values of lambda
colors = ("royalblue", "mediumaquamarine", "teal", "darkslategrey")

params=zip(lam,colors)
for param in params:
  y = []
  for i in x:
    P=ss.poisson(param[0])   # the Poisson distribution from scipy 
    y.append(P.pmf(i))
  plt.plot(x, y,
  label=r"$\lambda$ = %.1f" % param[0], 
           color = param[1])
  plt.fill_between(x, y, color = param[1], alpha = 0.3)

plt.legend(prop={'size':18});
plt.xlabel("$x$")
plt.ylabel("Probability Density", size = 18)
plt.ylim(top=1)
plt.title("Poisson Distributions", size = 20);

plt.show()         

![texto del enlace](https://docs.google.com/drawings/d/e/2PACX-1vS1BS7GNo0qZvY-pnS0_Nc0cYFQ8GloA32VJp0ISd9SS-e79lxnQg3PyGlNCzfTz5q2rGesy6eKAKnq/pub?w=902&h=376)

The Poisson distribution has the next properties:

*   Let X a random variable with Poisson distribution, then $E[X]=\lambda=Var(X)$, in other words, the mean of a random variable is equal to its variance.  
*   The Poisson distribution can be approximated by a Binomial distribution with adequate probability $p$ small enough and for $n$ big enough.

$$\begin{array}{ccc}\mathcal{B}_{n,p(n)}(k) & \overrightarrow{\small{n\to\infty}} & P_\lambda(k)\end{array}$$

***Observation:*** In our problem, the parameter $\lambda$ is unknown and will be estimated using Markov Chain Monte Carlo sampling, which is expose later.

# Bayesian approach

Before delving into Bayesian inference, we must keep in mind the basic notion of the Bayes theorem, which establishes a relationship between the probability that an event $\theta$ given $X$ will occur with the probability that $X$ given $\theta$ will occur, formally

$$P(\theta|X)=\frac{P(X|\theta)P(\theta)}{P(X)}$$

Where $P(\theta)$ is called prior probability and $P(\theta|X)$ is called the posterior probability.

The inferential approach is to interpret $X$ as a set of known data, and $\theta$ as an unknown value of interest (can be one or more parameters, missing measurements, ...) 

Our main goal is to find or estimate the value of $\theta$ via the final distribution 

$$p(\theta|x)$$

For this, we will postulate a probability model $p(x|\theta)$, a $p(\theta)$ distribution that adequately describes $\theta$ so that the initial uncertainty about its value is present and finally the $X$ data, which are conditioned to its observed value $x$, and then use the Bayes theorem and be able to conclude about our value $\theta$.

## Prior distribution for $\lambda$

Because we don't know a distribution for the average number of customer arrivals, $\lambda$, we can assume that its distribution is of the normal type.

The normal distribution belongs to the continuous distribution family; depends on two parameters: the average $\mu$ and the standard deviation $\sigma$, which are usually known as localization and scale parameters respectively; density function for normal distribution is given by  

$$f(x)=\frac{1}{\sigma\sqrt{2\pi}}\text{exp}\left[-\frac{1}{2}\left(\frac{x-\mu}{\sigma}\right)^2\right]\text{; }x\in\mathbb{R}$$

Probability density functions for four normal distributions are shown below.

In [ ]:
N = ss.norm    # the normal distribution from scipy
x = np.linspace(-10, 10, 1000)   # example domain of N
mu = (-3, -2, 0, 2)   # example means
sigma = (0.5, 0.75, 1, 1.5)   # example sd
colors = ("royalblue", "mediumaquamarine", "teal", "darkslategrey")

params = zip(mu, sigma, colors)
for param in params:
    y = N.pdf(x, loc = param[0], scale = param[1])
    plt.plot(x, y, 
             label="$\mu = %d,\;\\sigma = %.1f$" % (param[0], param[1]), 
             color = param[2])
    plt.fill_between(x, y, color = param[2], alpha = 0.3)
    
plt.legend(prop={'size':18});
plt.xlabel("$x$")
plt.ylabel("Probability Density", size = 18)
plt.xlim(left=-5,right=7.5)
plt.ylim(top=1)
plt.title("Normal Distributions", size = 20);

![texto del enlace](https://docs.google.com/drawings/d/e/2PACX-1vS87dFKM0v2UrBK05sgvQsj57Pvd3eWnRXfDf0AusKJ3toQxlSw9dgdcf1WMuWmuElQ-8htvpHHNLCF/pub?w=900&h=371)

# MCMC methods

## Markov chains

A markov chains is a sequence of random variables $\{X_n\}_n$ such that the markov property is satisfied, which tells us that

$$P(X_n=x_n|X_{n-1}=x_{n-1},X_{n-2}=x_{n-2},\cdots,X_0=x_0)=P(X_n=x_n|X_{n-1}=x_{n-1})$$

where $x_i$ is the state of $X_i$ in the iteration $i$.

The chains Markov can be classification in two types, homogeneous and inhomogeneous. The homogeneous Markov chains are chains Markov with the property what for all states $x_i$ and $x_j$, the probability of pass from state $x_i$ to state $x_j$ do not depends by time, in sense that

$$P(X_n=x_j|X_{n-1}=x_i)=P(X_1=x_j|X_0=x_i)$$

## What are MCMC methods?

Generally, MCMC's or Markov Chain Monte Carlo are methods for sampling distributions using Markov strings, mainly used in Bayesian inference and numerical integration.

MCMC's aim to build intelligently sampled strings, via a likelihood indicator, this indicator allows the choice of most likely samples in relation to the data set available to us.

## The Metropolis Hastings algorithm

The Metropolis Hastings algorithm belongs to the MCMC method set. It is used to produce random samples from a distribution where sampling would be difficult.

Generally, the algorithm generates a Markov chain with a stationary distribution, say $\pi$, in which each iteration consists of two steps: propose a new state of $\theta$ given a status of $\theta_k$ and accept or reject it, according to the principle of plausibility, which tells us that the values of $\theta$ that assign a high probability to observation $x$ are more plausible than the values of $\theta$ that assign a small probability to $x$.

Before enunciating the Metropolis hasting algorithm, we must propose a density function $q$, which will act as a way to move randomly, through the space of possible values of $\theta$, as we perform the process of finding the acceptance values, knowing the current state of each iteration.

### The algorithm MH

1.   Given a value of $\theta_k$, simulate a $\theta$ candidate of a density $q(\theta|\theta_k)$.
2.   Calculate the probability of acceptance for the generated value $\theta$

$$\alpha=\text{min}\left\{1,\frac{\pi(\theta|x)q(\theta_k|\theta)}{\pi(\theta_k|x)q(\theta|\theta_k)}\right\}$$

3.   Simulate a value $u$ of a uniform distribution  $\mathcal{U}(0,1)$.
4.   Si $u<\alpha$, take $\theta_{k+1}=\theta$, otherwise take $\theta_{k+1}=\theta_k$.
5.   Go back to 1.





# Posterior probability for the arrival of people

We now have all the tools to determine the subsequent probability $P(\theta|X)$. Although we don't yet know the value of the $\lambda$ parameter, we'll use Poisson's distribution to describe customers' arrival at the Burger Dome. We will use the MCMC methods to determine the value of $\lambda$ based on the assumption that it is normally distributed with average $\mu$ and standard deviation $\sigma$.

Python gives us an easy way to implement MCMC methods. The PyMC3 library is a powerful tool to implement these and other inferential algorithms. In the following code we will implement the MH algorithm, which will draw an arbitrary number of samples for $\lambda$.

In [ ]:
# number of samples
N_samples = 50000 
 
# the array with the number of arrivals by time interval
arrivals_obs = np.array(counts)  

In [ ]:
# the Burger Dome model 
with pm.Model() as burger_model:
  # create the parameter lambda
  lam = pm.Normal('$\lambda$', mu=0.001, sigma=1)

  # the Poisson distribution for the probability
  p = pm.Poisson('p',lam, observed=arrivals_obs)

  # using Metropolis Hastings Sampling
  step = pm.Metropolis()

  # sample from the posterior using the sampling method
  burger_trace = pm.sample(N_samples, step=step);

Sequential sampling (2 chains in 1 job)

Metropolis: [\$\lambda\$]

Sampling chain 0, 0 divergences: 100%|██████████| 50500/50500 [00:08<00:00, 6254.23it/s]

Sampling chain 1, 0 divergences: 100%|██████████| 50500/50500 [00:07<00:00, 6870.61it/s]

The number of effective samples is smaller than 10% for some parameters.

The variable burger_trace contains all the samples of $\lambda$, the idea of MCMC methods is that as the algorithm iterates, the samples become more likely. Next we will perform a histogram that will allow us to inspect the behavior of the samples:

In [ ]:
# Extract the lambda samples
lam_samples = burger_trace["$\lambda$"][50000:, None]

plt.subplot(211)
plt.title(r"""Distribution of $\lambda$ with %d samples""" % N_samples)
plt.hist(lam_samples, histtype='stepfilled',color="darkolivegreen",
         bins=30, alpha=0.8, density=True);
plt.ylabel('Probability Density')
plt.show()

![texto del enlace](https://docs.google.com/drawings/d/e/2PACX-1vQLc7UiF8iwWVbsHFoHUuCBAl2BjFqCoc1fdphfzOoWRkL_s-Jqz0DoHh4w5PoygW04t_YHM6ryz6TZ/pub?w=901&h=191)

It is important to mention that the value we take as an initial condition can alter the convergence speed of our algorithm, usually considered a relatively large amount of iterations to ensure convergence, however, as our goal is to make inferences from the accepted values, the information present in the $50\%$ can cause biases in our predictions , for this reason we can find examples in which this data is simply extracted from the analysis performed. These values are known as burned values or burn values.

In [ ]:
# Trace for lambda values
plt.subplot(211)
plt.title(r'Distribution of $\lambda$')
plt.plot(lam_samples[int(len(lam_samples)/2):], 
         color = 'darkslategrey', alpha=0.8)
plt.xlabel('Samples'); plt.ylabel('Parameter');

![texto del enlace](https://docs.google.com/drawings/d/e/2PACX-1vRn2TlPESjksLbwLYYG0-6wk7eu9jUwRA_FF2pczVGhIjg24tptYWz6Bw3Z8gS9eaMrePrKv2_Sg8-Q/pub?w=900&h=208)

We can see that the most likely values for $\lambda$ are around $0.4$, this we can interpret as that every 15 minutes, on average $0.4\cdot(15\text{ minutes})=6$ customers arrive, this because according to the observations and the model implemented, it is less likely that $0.35\cdot15\approx5$ people will enter. To find the most likely subsequent distribution for the arrival of customers at the Burger Dome, we will take the average samples of $\lambda$.

In [ ]:
# samples with no burning values
lam_samples = lam_samples[int(len(lam_samples)/2):]

# Consider the most probable parameter as the mean value
lam_est = lam_samples.mean()

# Probability at each time using mean value of lambda
entradas_est = ss.poisson(lam_est).pmf(arrivals_est)

print("On average, every minute %.3f people arrive" %lam_est)

On average, every minute 0.408 people arrive

In [ ]:
figsize(16,6)
# Poisson model with Teacher's parameter
entradas_est1 = ss.poisson(0.5).pmf(arrivals_est)  

# most probable Poisson model
plt.plot(arrivals_est, entradas_est, color = 'navy', 
         lw=3, label="Most probable Poisson model")
plt.plot(arrivals_est, entradas_est1,'r--',
         lw=3, label="Poisson model with $\lambda$=0.5")

# histogram for the number of arrivals per time interval
#plt.hist(counts, density=True, label="Obs",bins=3)
plt.bar(arrivals_est,count_norm,label="obs")

plt.legend(prop={'size':18})
plt.xlabel("People entering the system at %d minute intervals" %n)
plt.ylabel("Frequency")
plt.title("Probability distribution of arrivals with %d samples" %N_samples)
plt.ylim(top=1)

plt.show()

![texto del enlace](https://docs.google.com/drawings/d/e/2PACX-1vQp27yc09wZsKcrXoMoyH_KsC2zcnA6Pgt6-9SkFs1wxpL-INESATuQRWJ16h6zvDlOGLloKUO_PVvn/pub?w=904&h=375)

Posterior allows us to know the probability for the arrival of an n number of people per minute

In [ ]:
print('The probability of a person coming: {:.2f}%.'.
      format(100 * ss.poisson(lam_est).pmf(1)))
print('The probability of three people arriving: {:.2f}%.'.
      format(100 * ss.poisson(lam_est).pmf(3)))
print('The probability that no one will arrive: {:.2f}%.'.
      format(100 * ss.poisson(lam_est).pmf(0)))

The probability of a person coming: 27.14%.

The probability of three people arriving: 0.75%.

The probability that no one will arrive: 66.49%.

We know that there is considerable uncertainty in the estimates for $\lambda$. To reflect this on the chart, we can include a 95% confidence interval at each time based on all samples.

In [ ]:
burger_all_est = ss.poisson(lam_samples).pmf(arrivals_est)
quantiles = ss.mstats.mquantiles(burger_all_est, [0.025, 0.975], axis=0)

In [ ]:
*plt.fill_between(arrivals_est, quantiles, alpha=0.7, 
                 color='steelblue', label = '95% CI')
plt.plot(arrivals_est, entradas_est, color = 'navy', ls="--", 
         lw=3, label="Most probable Poisson model")
#plt.hist(counts, density=True, bins = 5)
plt.scatter(arrivals_est,count_norm,label="obs",color="lime")
plt.legend(prop={'size':14})
plt.xlabel('Number of arrivals every %d minutes' %n) ; plt.ylabel('Probability'); 
plt.title('Posterior Probabilty with 95% CI');

![texto del enlace](https://docs.google.com/drawings/d/e/2PACX-1vR9DFzcQlrsGWVONKPq4HtuUisxZMOqWZnhCKrJhMvNnIFp0Xh6OK9aurW5E8akTHFpN0oCgVCMxpPL/pub?w=908&h=369)

We can assure you that with 95% confidence that our observations will fall into the blue margin, this allows us to observe that there is some uncertainty with the parameter generated by the MH algorithm.

We can also plot the subsequent distribution of arrivals as a histogram based on all samples for the parameters. 

In [ ]:
def burger_posterior(n_people):
  figsize(16, 8)
  prob = ss.poisson(lam_samples).pmf(n_people)
  plt.hist(prob, bins=100, histtype='step', lw=4)
  plt.title('Probability distribution for the arrival of %d people' % n_people)
  plt.xlabel('Probability of arrivals'); plt.ylabel('Samples')
  plt.show()

In [ ]:
burger_posterior(0)

![texto del enlace](https://docs.google.com/drawings/d/e/2PACX-1vQna41i0Xr0-EjC2vyJxFuPeUljHnTIAhBPzhY_F9pONhI_T-TZGxqowVXHdo6tQY29tTus2BJNlOZT/pub?w=884&h=452)

In [ ]:
burger_posterior(2)

![texto del enlace](https://docs.google.com/drawings/d/e/2PACX-1vS5b2j684gQsa3uYe-d8gU5B6pEhnuv_6B1oV3Zam3l90Q0OCni9P0sY9xc5LYFpbqRD75H3RYjoLWb/pub?w=881&h=451)

In [ ]:
burger_posterior(5)

![texto del enlace](https://docs.google.com/drawings/d/e/2PACX-1vSTC6fij5h74zmoXMaAfar5x0t44uz3pQeyUFi87XhZ8MLPGcbAEHTUlmPdvW5u14m6QWHLRxTT1u17/pub?w=881&h=451)

## Goodness and fit test

Now we will verify that the $\lambda$ parameter that we generate from the MH algorithm properly describes the data, for this we will perform a goodness and adjustment test of chi squared for discrete distributions. According to [7], we will use groupings so that the expected amount of data in each group is greater than or equal to 5:

In [ ]:
# Tags
num_arr = []
for i in range(3):
  num_arr.append(str(i))
num_arr.append("3 or more")

# Frecuency
counts2 = []
for i in range(3):
  counts2.append(counts.count(i))
counts2.append(counts.count(3)+counts.count(4))

pd.DataFrame({"Number of arrivals":num_arr, "Frequency":counts2})

![texto del enlace](https://docs.google.com/drawings/d/e/2PACX-1vR3WPWTk3V40M_FF7DUShi3VQSCx-cSSgQ9_SX00fzd4yUAXA1Hw_PJrZ-zkyM1CWGeVzs28lsQncHj/pub?w=242&h=140)

We will assume that the data follow a Poisson distribution, so we will perform the hypothesis test trying to verify whether the data affirm a conclusion contrary to our assumption, In formal terms, we will try to show that the statistics $\mathcal{X}^2\leq\mathcal{X}^2_{\alpha,k-1-m}$, where $k$ represents the number of groups, $m$ the number of parameters in the model, $\alpha$ the significance of the test and $\mathcal{X}^2_{\alpha,k-1-m}$ the value in the table for the square chi distribution with $k-1-m$ degrees of freedom. In our case, $m=1$ and $k=4$.

Next, we'll calculate the estimated amounts for $\hat{\lambda}=\text{lam_est}$.

In [1]:
est_am = []

for i in range(3):
  est_am.append(len(counts)*ss.poisson(lam_est).pmf(i))
est_am.append(len(counts)-sum(est_am))

pd.DataFrame({"Number of arrivals":num_arr, "Frequency":counts2, "Estimated amounts": est_am})

![texto del enlace](https://docs.google.com/drawings/d/e/2PACX-1vRaMpiXtK_gwUJyegi2bW9HsaDKor1M8pk-p3E5yR3WlpX0sBAcnk-y8iAmIKUSC6uuJA-Zqc9-v0n3/pub?w=374&h=140)

we'll finally calculate the value of $\mathcal{X}^2$ and the p-value:

In [1]:
D,p_value = ss.chisquare(counts2,est_am)   # scipy chi square
print("The value of the statistic is %.4f and the p-value of the test is %.4f" %(D,p_value))

The value of the statistic is 0.2217 and the p-value of the test is 0.9740

Therefore, we can say that the Poisson distribution appropriately describes the distribution of the data, since the lowest supported significance, to reject $H_0$ is $97.40\%$, which is an exaggeratedly large value for $\alpha$.

# Conclusions

Based on the data and the tests carried out in this letter, we can conclude that the arrival of people at the Burger Dome effectively follows a distribution of Poisson, with average $\lambda=0.40765$, this implies that on average every minute $0.40765$ people arrive at the restaurant or, equal to that on average, every $2$ minutes with $46$ seconds (approximately $1/\lambda$), a person arrives.

# References

[1]   Probabilidad y estadística, Liliana Blanco, Universidad Nacional de Colombia, 2004.

[2]   Markov_chain_monte_carlo, WillKoehrsen, Github, 2018.

[3]   Métodos de cadenas de Markov Monte Carlo, Conchi Ausín, Universidad Carlos III de Madrid, 2012.

[4]   Estadística Bayesiana - Teoría y Conceptos Básicos, Eduardo Gutiérrez Peña, Universidad Nacional Autonoma de México.

[5]   MCMC Methods for data modeling, Kenneth Scerri.

[6]   The Metropolis Hastings Algorithm, Matthew Stephens, 2018.

[7]   Probabilidad y estadística para ingeniería y ciencias básicas, Jay L. Devore, California Polytechnic State University, 2008.
